In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import pandas as pd
import random
import re
from tqdm import tqdm
import copy

# ========================================
# 1. Load Model
# ========================================
model_name = "Qwen/Qwen3-0.6B"

print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# ========================================
# 2. Load and prepare datasets (10%)
# ========================================
print("Loading datasets...")

mmlu = load_dataset("cais/mmlu", "all", split="test")
gsm8k = load_dataset("gsm8k", "main", split="test")

mmlu_10 = mmlu.shuffle(seed=42).select(range(int(0.1 * len(mmlu))))
gsm8k_10 = gsm8k.shuffle(seed=42).select(range(int(0.1 * len(gsm8k))))

# ========================================
# 3. MMLU Evaluation (MCQ)
# ========================================
def eval_mmlu(model, tokenizer, dataset):
    correct = 0

    for row in tqdm(dataset, desc="Evaluating MMLU"):
        question = row["question"]
        options = row["choices"]
        answer = row["answer"]

        prompt = "Answer the following multiple-choice question.\n\n"
        prompt += f"Question: {question}\n"
        for i, c in enumerate(options):
            prompt += f"{chr(65+i)}. {c}\n"
        prompt += "\nAnswer:"

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        output = model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False
        )

        response = tokenizer.decode(output[0], skip_special_tokens=True)
        pred = response.strip()[-1:].upper()

        if pred == answer:
            correct += 1

    return correct / len(dataset)


# ========================================
# 4. GSM8K Evaluation (open-ended math)
# ========================================
def extract_last_number(text):
    nums = re.findall(r"-?\d+\.?\d*", text)
    return nums[-1] if nums else None

def eval_gsm8k(model, tokenizer, dataset):
    correct = 0

    for row in tqdm(dataset, desc="Evaluating GSM8K"):
        question = row["question"]
        gold_answer = row["answer"]

        prompt = (
            "Solve the following math word problem. "
            "Give only the final numeric answer.\n\n"
            f"{question}\n\nAnswer:"
        )

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        output = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=False
        )

        response = tokenizer.decode(output[0], skip_special_tokens=True)

        pred = extract_last_number(response)
        gold = extract_last_number(gold_answer)

        if pred == gold:
            correct += 1

    return correct / len(dataset)


# ========================================
# 5. STATIC PRUNING LOOP
# ========================================
layers = base_model.model.layers
num_layers = len(layers)

print(f"\nModel has {num_layers} transformer layers.\n")

results = []

for prune_layer in range(num_layers):
    print(f"\n===== TESTING WITH LAYER {prune_layer} REMOVED =====")

    # clone the original model each time
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # === REMOVE THE TARGET LAYER ===
    pruned_layers = torch.nn.ModuleList(
        [layer for i, layer in enumerate(model.model.layers) if i != prune_layer]
    )

    model.model.layers = pruned_layers
    print(f"Layer {prune_layer} removed. New layer count: {len(model.model.layers)}")

    # === Evaluate ===
    mmlu_score = eval_mmlu(model, tokenizer, mmlu_10)
    gsm8k_score = eval_gsm8k(model, tokenizer, gsm8k_10)

    print(f"Layer {prune_layer}: MMLU={mmlu_score:.4f}, GSM8K={gsm8k_score:.4f}")

    results.append({
        "removed_layer": prune_layer,
        "mmlu_accuracy": mmlu_score,
        "gsm8k_accuracy": gsm8k_score
    })


# ========================================
# 6. Save results
# ========================================
df = pd.DataFrame(results)
df.to_csv("layer_pruning_results.csv", index=False)

print("\nSaved layer pruning results to: layer_pruning_results.csv")


Loading model...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Layer 25: MMLU=0.0000, GSM8K=0.0840

===== TESTING WITH LAYER 26 REMOVED =====
Layer 26 removed. New layer count: 27


Evaluating GSM8K: 100%|██████████| 131/131 [14:12<00:00,  6.51s/it]


Layer 26: MMLU=0.0000, GSM8K=0.0763

===== TESTING WITH LAYER 27 REMOVED =====
Layer 27 removed. New layer count: 27


Evaluating GSM8K: 100%|██████████| 131/131 [14:10<00:00,  6.49s/it]

Layer 27: MMLU=0.0000, GSM8K=0.0840

Saved layer pruning results to: layer_pruning_results.csv
